# Interactive visualisation of the prediction for a single subject

In [ ]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from random import randint
import pandas as pd
import pickle
from preprocessing.preprocessing_tools.normalisation.reverse_normalisation import reverse_normalisation
from prediction.utils.shap_helper_functions import check_shap_version_compatibility
from prediction.utils.utils import flatten
from prediction.outcome_prediction.data_loading.data_loader import load_data
from prediction.utils.utils import aggregate_features_over_time
from sklearn.model_selection import train_test_split
from prediction.outcome_prediction.data_loading.data_formatting import format_to_2d_table_with_time, \
    link_patient_id_to_outcome, features_to_numpy, numpy_to_lookup_table

In [ ]:
# Shap values require very specific versions
check_shap_version_compatibility()

In [ ]:
features_path = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/preprocessed_features_01012023_233050.csv'
labels_path = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/preprocessed_outcomes_01012023_233050.csv'
normalisation_parameters_path = '/Users/jk1/temp/opsum_prepro_output/gsu_prepro_01012023_233050/logs_01012023_233050/normalisation_parameters.csv'
shap_over_time_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/3M_mrs02/with_feature_aggregration/testing/tree_explainer_shap_values_over_ts.pkl'
predictions_over_time_path = '/Users/jk1/temp/opsum_prediction_output/linear_72h_xgb/with_feature_aggregration/testing/predictions_over_timesteps.pkl'
out_dir = '/Users/jk1/Downloads'

In [ ]:
outcome = '3M mRS 0-2'
test_size = 0.2
seed = 42
n_splits = 5
moving_average = False
n_time_steps = 72
total_n_features = 84

In [ ]:
n_features = 3

In [ ]:
with open(shap_over_time_path, 'rb') as handle:
    shap_values_over_time = pickle.load(handle)
shap_values_over_time = np.array(shap_values_over_time)
normalisation_parameters_df = pd.read_csv(normalisation_parameters_path)

In [ ]:
with open(predictions_over_time_path, 'rb') as handle:
    predictions_over_time = pickle.load(handle)
predictions_over_time = np.array(predictions_over_time)

## Load data

In [ ]:
X, y = format_to_2d_table_with_time(feature_df_path=features_path, outcome_df_path=labels_path,
                                    outcome=outcome)

# Reduce every patient to a single outcome (to avoid duplicates)
all_pids_with_outcome = link_patient_id_to_outcome(y, outcome)
pid_train, pid_test, y_pid_train, y_pid_test = train_test_split(all_pids_with_outcome.patient_id.tolist(),
                                                                all_pids_with_outcome.outcome.tolist(),
                                                                stratify=all_pids_with_outcome.outcome.tolist(),
                                                                test_size=test_size,
                                                                random_state=seed)
# Preprocess overall train data
train_X_df = X[X.patient_id.isin(pid_train)]
train_y_df = y[y.patient_id.isin(pid_train)]
train_X_np = features_to_numpy(train_X_df,
                               ['case_admission_id', 'relative_sample_date_hourly_cat', 'sample_label', 'value'])
train_y_np = np.array([train_y_df[train_y_df.case_admission_id == cid].outcome.values[0] for cid in
                       train_X_np[:, 0, 0, 0]]).astype('float32')
train_X_np = train_X_np[:, :, :, -1].astype('float32')


# Preprocess overall test data
test_X_df = X[X.patient_id.isin(pid_test)]
test_y_df = y[y.patient_id.isin(pid_test)]
test_X_np = features_to_numpy(test_X_df,
                              ['case_admission_id', 'relative_sample_date_hourly_cat', 'sample_label', 'value'])
test_y_np = np.array([test_y_df[test_y_df.case_admission_id == cid].outcome.values[0] for cid in
                      test_X_np[:, 0, 0, 0]]).astype('float32')
# create look-up table for case_admission_ids, sample_labels and relative_sample_date_hourly_cat
test_features_lookup_table = numpy_to_lookup_table(test_X_np)
# Remove the case_admission_id, sample_label, and time_step_label columns from the data
test_X_np = test_X_np[:, :, :, -1].astype('float32')

In [ ]:
X_test, y_test = aggregate_features_over_time(test_X_np, test_y_np, moving_average=moving_average)
X_test = X_test.reshape(-1, n_time_steps, X_test.shape[-1]).astype('float32')

In [ ]:
original_features = list(test_features_lookup_table['sample_label'])

In [ ]:
avg_feature_names = [f'avg_{item}' for item in list(test_features_lookup_table['sample_label'])]
min_feature_names = [f'min_{item}' for item in list(test_features_lookup_table['sample_label'])]
max_feature_names = [f'max_{item}' for item in list(test_features_lookup_table['sample_label'])]

feature_names = flatten([original_features, avg_feature_names, min_feature_names, max_feature_names])

In [ ]:
non_normalised_train_X_df = reverse_normalisation(train_X_df, normalisation_parameters_df)

## Choose subject and load prediction

In [ ]:
subj = randint(0, len(test_X_np))

In [ ]:
print(subj, predictions_over_time[-1,subj], y_test[subj])

In [ ]:
subj_pred_over_ts = predictions_over_time[:,subj]

## Plot overall subject prediction & explanation

In [ ]:
n_features = 3

In [ ]:
from prediction.utils.visualisation_helper_functions import reverse_normalisation_for_subj

# plot a bar plot showing impact of most important features on the prediction across all n_time_steps
# find index of 3 features with biggest positive shap impart
selected_positive_features = np.squeeze(shap_values_over_time[-1])[subj].argsort()[-n_features:][::-1]

# find index of 3 features with biggest negative shap impart
selected_negative_features = np.squeeze(shap_values_over_time[-1])[subj].argsort()[:n_features][::-1]

selected_features = np.concatenate((selected_positive_features, selected_negative_features))

fig1 = plt.figure(figsize=(15,5))
ax1 = fig1.add_subplot(121)
ax = sns.barplot(y=np.array(feature_names)[selected_features], x=np.squeeze(shap_values_over_time[-1])[subj][selected_features], palette="RdBu_r")
ax.title.set_text(f'SHAP values for subj {subj} ')

non_norm_subj_df = reverse_normalisation_for_subj(pd.DataFrame(data=test_X_np[subj], columns = original_features), normalisation_parameters_df)
# display median of original feature value (and not aggregation) - therefore modulo is taken
median_norm_feature_df = non_norm_subj_df.median(axis=0)[selected_features % total_n_features]

ax2 = fig1.add_subplot(122)
font_size=12
bbox=[0, 0, 1, 1]
ax2.axis('off')
cell_text = []
for row in range(len(median_norm_feature_df)):
    cell_text.append([median_norm_feature_df.iloc[row].astype(str)])
mpl_table = ax2.table(cellText = cell_text, rowLabels = median_norm_feature_df.index, bbox=bbox, colLabels=['Normalised value'], cellLoc='center', colLoc='center', loc='center')
mpl_table.auto_set_font_size(False)
mpl_table.set_fontsize(font_size)

fig1.set_tight_layout(True)
# set figure title
fig1.suptitle(f'Explanation of prediction for subj {subj} with a probability of good outcome of {subj_pred_over_ts[-1]:.2f}', fontsize=20)

plt.show()

In [ ]:
# fig1.savefig(os.path.join(out_dir, 'final_prediction.png'), dpi=600)

## Plot relevant features in relation to training population

In [ ]:
fig2 = plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.2)
plt.suptitle("Selected features", fontsize=18, y=0.99, x=0.52, horizontalalignment='center')

# set number of columns (use 3 to demonstrate the change)
ncols = 3
# calculate number of rows
nrows = len(selected_features) // ncols + (len(selected_features) % ncols > 0)

# loop through the length of features and keep track of index
for n, feature in enumerate(selected_features):
    # add a new subplot iteratively using nrows and cols
    ax = plt.subplot(nrows, ncols, n + 1)

    temp_pop_df = non_normalised_train_X_df[non_normalised_train_X_df.sample_label == original_features[feature % total_n_features]]
    sns.histplot(temp_pop_df.value, ax=ax)
    plt.scatter(median_norm_feature_df[original_features[feature % total_n_features]], 0, marker='o', s=500)
    if (n % ncols) == 1:
        if n <= len(selected_features) / 2:
            ax.set_title(r"$\bf{Positive\ features}$" +f'\n\n{original_features[feature % total_n_features]}')
        else:
            ax.set_title(r"$\bf{Negative\ features}$" + f'\n\n{original_features[feature % total_n_features]}')

    else:
        ax.set_title(original_features[feature % total_n_features])

plt.tight_layout()

In [ ]:
# fig2.savefig(os.path.join(out_dir, 'features_histogram_comparison.png'), dpi=600)

In [ ]:
def density_jitter(values, width=1.0, cluster_factor=1.0):
    """
    Add jitter to a 1D array of values, using a kernel density estimate
    """
    inds = np.arange(len(values))
    np.random.shuffle(inds)
    values = values[inds]
    N = len(values)
    nbins = 100
    quant = np.round(nbins * (values - np.min(values)) / (np.max(values) - np.min(values) + 1e-8))
    inds = np.argsort(quant + np.random.randn(N) * 1e-6)
    layer = 0
    last_bin = -1
    ys = np.zeros(N)
    for ind in inds:
        if quant[ind] != last_bin:
            layer = 0
        ys[ind] = cluster_factor * (np.ceil(layer / 2) * ((layer % 2) * 2 - 1))
        layer += 1
        last_bin = quant[ind]
    ys *= 0.9 * (width / np.max(ys + 1))

    return ys

In [ ]:
from matplotlib.legend_handler import HandlerTuple

fig2_5 = plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.2)
plt.suptitle("Selected features", fontsize=18, y=0.99, x=0.52, horizontalalignment='center')

overlay_population_scatter = True
overlay_subject_scatter = False
plot_legend = True
shap_value_base = 1000
label_font_size = 12

# set number of columns (use 3 to demonstrate the change)
ncols = n_features
# calculate number of rows
nrows = 2

# loop through the length of features and keep track of index
for n, feature in enumerate(selected_features):
    # add a new subplot iteratively using nrows and cols
    ax = plt.subplot(nrows, ncols, n + 1)

    temp_pop_df = non_normalised_train_X_df[non_normalised_train_X_df.sample_label == original_features[feature % total_n_features]].groupby('case_admission_id').median()
    sns.violinplot(temp_pop_df.value, ax=ax, color=feature_color_dict[feature], alpha=0.1, inner=None)
    plt.setp(ax.collections, alpha=.1)

    if overlay_population_scatter:
        ys = density_jitter(temp_pop_df.value.values, width=0.25, cluster_factor=1)
        ax.scatter(0 + ys, temp_pop_df.value, alpha=0.005, color='grey')

    if overlay_subject_scatter:
        ys = density_jitter(non_norm_subj_df[np.array(feature_names)[feature % total_n_features]].values, width=0.5)
        plt.scatter(0 + ys, non_norm_subj_df[np.array(feature_names)[feature % total_n_features]].values, marker='o', zorder=10, color='grey', alpha=0.1)

    # set weight of marker based on shap value
    marker_size = np.abs(shap_values_over_time[-1, subj, feature]) * shap_value_base

    # set z-order to make sure the scatter plot is on top
    plt.scatter(0, median_norm_feature_df[original_features[feature % total_n_features]], marker='o', s=marker_size, zorder=10, color=feature_color_dict[feature], alpha=0.8)
    if (n % ncols) == 1:
        if n <= len(selected_features) / 2:
            ax.set_title(r"$\bf{Positive\ features}$" +f'\n\n{original_features[feature % total_n_features]}')
        else:
            ax.set_title(r"$\bf{Negative\ features}$" + f'\n\n{original_features[feature % total_n_features]}')

    else:
        ax.set_title(original_features[feature % total_n_features])

    ax.set_xlim(-1, 1)
    ax.set_ylim(0, temp_pop_df.value.max() + temp_pop_df.value.max() / 5)
    # turn off x axis
    # ax.set_xticks([])

    # add a legend for the shap value marker size on the bottom right of the last plot
    if (n == len(selected_features) /2 - 1) & plot_legend:
        legend_markers, legend_labels = ax.get_legend_handles_labels()
        subj_value_1 = plt.scatter([], [], marker='o', s=shap_value_base/4, color=feature_color_dict[selected_features[0]], alpha=0.8)
        subj_value_2 = plt.scatter([], [], marker='o', s=shap_value_base/8, color=feature_color_dict[selected_features[1]], alpha=0.8)
        subj_value_3 = plt.scatter([], [], marker='o', s=shap_value_base/16, color=feature_color_dict[selected_features[2]], alpha=0.8)
        subj_marker = (subj_value_1, subj_value_2, subj_value_3)
        subj_labels = 'Subject feature value\n(Size proportional to\nweight in model)'
        legend_markers.append(subj_marker)
        legend_labels.append(subj_labels)

        violin_patch1 = mpatches.Patch(color=feature_color_dict[selected_features[0]], alpha=0.1)
        violin_patch2 = mpatches.Patch(color=feature_color_dict[selected_features[1]], alpha=0.1)
        violin_patch3 = mpatches.Patch(color=feature_color_dict[selected_features[2]], alpha=0.1)
        violin_plot_marker = (violin_patch1, violin_patch2, violin_patch3)
        violin_plot_label = 'Distribution in\ntraining population'
        legend_markers.append(violin_plot_marker)
        legend_labels.append(violin_plot_label)


        if overlay_population_scatter:
            # add a legend for the violin contour plots (population)
            legend_markers.append(plt.scatter([], [], marker='o', s=10, color='grey', alpha=0.3))
            legend_labels.append('Individual values in\ntraining population')

        # plot legend outside of right side of plot
        # and avoid that markers are on the legend box border
        ax.legend(legend_markers, legend_labels, fontsize=label_font_size,
                  handler_map={tuple: HandlerTuple(ndivide=None)},
                    bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.,
                    handleheight=2, handlelength=4)

plt.tight_layout()


## Plot evolution of prediction & explanation over time

In [ ]:
overall_prevailing_features = False
weigh_by_feature_value = False

In [ ]:
cumulative_shap_values_over_time = np.array([shap_values_over_time[ts].sum(axis=1) for ts in range(n_time_steps)])

# find index of 3 features with biggest positive shap impart & index of 3 features with biggest negative shap impart
if overall_prevailing_features:
    # prevailing features over cumulative time
    selected_negative_features = cumulative_shap_values_over_time[:, subj].argsort()[:n_features][::-1]
    selected_positive_features = cumulative_shap_values_over_time[:, subj].argsort()[-n_features:][::-1]
else:
    # prevailing features at last timepoint
    selected_positive_features = np.squeeze(shap_values_over_time[-1])[subj].argsort()[-n_features:][::-1]
    selected_negative_features = np.squeeze(shap_values_over_time[-1])[subj].argsort()[:n_features][::-1]

selected_features = np.concatenate((selected_positive_features, selected_negative_features))

fig3 = plt.figure(figsize=(15,10))

k=0.05
alpha=0.3

timestep_axis = np.array(range(n_time_steps))

positive_color_palette = sns.color_palette("mako", n_colors=len(selected_positive_features))
negative_color_palette = sns.color_palette("flare_r", n_colors=len(selected_negative_features))

timestep_axis = np.array(range(n_time_steps))
ax = sns.lineplot(x=timestep_axis, y=subj_pred_over_ts, label='probability', linewidth = 2)


pos_baseline = subj_pred_over_ts
neg_baseline = subj_pred_over_ts
pos_count = 0
neg_count = 0
feature_color_dict = {}

for i, feature in enumerate(selected_features):
    subj_feature_shap_value_over_time = shap_values_over_time[:, subj, feature]
    positive_portion = (subj_feature_shap_value_over_time > 0)
    negative_portion = (subj_feature_shap_value_over_time < 0)

    pos_function = subj_feature_shap_value_over_time.copy()
    pos_function[negative_portion] = 0

    neg_function = subj_feature_shap_value_over_time.copy()
    neg_function[positive_portion] = 0

    if feature in selected_positive_features:
        feature_color = positive_color_palette[pos_count]
        pos_count += 1
    else:
        feature_color = negative_color_palette[neg_count]
        neg_count += 1
    feature_color_dict[feature] = feature_color

    positive_feature = pos_baseline + k * pos_function
    if weigh_by_feature_value:
        positive_feature *= X_test[subj, :, feature] / X_test[:, :, feature].max()
    ax.fill_between(timestep_axis, pos_baseline, positive_feature, color=feature_color, alpha=alpha, label=feature_names[feature])
    pos_baseline = positive_feature

    negative_feature = neg_baseline + k * neg_function
    if weigh_by_feature_value:
        negative_feature *= X_test[subj, :, feature] / X_test[:, :, feature].max()
    ax.fill_between(timestep_axis, negative_feature, neg_baseline, color=feature_color, alpha=alpha)
    neg_baseline = negative_feature

ax.legend(fontsize='x-large')

ax.set_title(f'Predictions for subject {subj} of test set along time', fontsize=20)
ax.set_xlabel('Time from admission (hours)', fontsize=15)
ax.set_ylabel('Probability of favorable outcome', fontsize=15)

# ax.set_ylim(0,1)

plt.plot()



In [ ]:
# fig3.savefig(os.path.join(out_dir, 'prediction_over_time.png'), dpi=600)

## Plot selected features over time

In [ ]:
fig4 = plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.2)
plt.suptitle("Selected features", fontsize=18, y=0.99, x=0.52, horizontalalignment='center')

# set number of columns (use 3 to demonstrate the change)
ncols = 3
# calculate number of rows
nrows = len(selected_features) // ncols + (len(selected_features) % ncols > 0)

pos_count = 0
neg_count = 0
# loop through the length of features and keep track of index
for n, feature in enumerate(selected_features):
    # add a new subplot iteratively using nrows and cols
    ax = plt.subplot(nrows, ncols, n + 1)

    if feature in selected_positive_features:
        feature_color = positive_color_palette[pos_count]
        pos_count += 1
    else:
        feature_color = negative_color_palette[neg_count]
        neg_count += 1

    sns.lineplot(y=original_features[feature % total_n_features], x='index', data=non_norm_subj_df.reset_index(), color=feature_color, ax=ax)

    if (n % ncols) == 1:
        if n <= len(selected_features) / 2:
            ax.set_title(r"$\bf{Positive\ features}$" +f'\n\n{original_features[feature % total_n_features]}')
        else:
            ax.set_title(r"$\bf{Negative\ features}$" + f'\n\n{original_features[feature % total_n_features]}')
    else:
        ax.set_title(feature_names[feature])
plt.tight_layout()


In [ ]:
# fig4.savefig(os.path.join(out_dir, 'features_over_time.png'), dpi=600)

## Identify features driving changes in prediction over time

In [ ]:
threshold = 0.02
n_features = 0
k=0.05
alpha=0.3

# identify significant changes in prediction over time by a change in threshold X% in prediction
significant_timesteps = np.where(np.abs(np.diff(subj_pred_over_ts)) > threshold)[0]
# for each timestep, identify the feature that has the largest impact on the prediction
selected_features_by_impact = np.abs(np.diff(shap_values_over_time[:, subj], axis=0))[significant_timesteps].argmax(axis=1)
selected_positive_features_by_impact = [selected_features_by_impact[ftx] for ftx in range(len(significant_timesteps))
  if shap_values_over_time[significant_timesteps[ftx] + 1, subj, selected_features_by_impact[ftx]] > 0]
selected_negative_features_by_impact = [selected_features_by_impact[ftx] for ftx in range(len(significant_timesteps))
  if shap_values_over_time[significant_timesteps[ftx] + 1, subj, selected_features_by_impact[ftx]] < 0]

if n_features == 0:
    selected_positive_features = np.array([])
    selected_negative_features = np.array([])
else:
    selected_positive_features = shap_values_over_time[-1,subj].argsort()[-n_features:][::-1]
    selected_negative_features = shap_values_over_time[-1,subj].argsort()[:n_features][::-1]

selected_features = np.concatenate((selected_positive_features, selected_positive_features_by_impact, selected_negative_features, selected_negative_features_by_impact)).astype(int)

fig3 = plt.figure(figsize=(15,10))


positive_color_palette = sns.color_palette("mako", n_colors=len(set(np.concatenate((selected_positive_features, selected_positive_features_by_impact)))))
negative_color_palette = sns.color_palette("flare_r", n_colors=len(set(np.concatenate((selected_negative_features, selected_negative_features_by_impact)))))

# plot prediction over time
timestep_axis = np.array(range(n_time_steps))
ax = sns.lineplot(x=timestep_axis, y=subj_pred_over_ts, label='probability', linewidth = 2)

pos_baseline = subj_pred_over_ts
neg_baseline = subj_pred_over_ts
pos_count = 0
neg_count = 0
feature_color_dict = {}
for i, feature in enumerate(set(selected_features)):
    subj_feature_shap_value_over_time = shap_values_over_time[:, subj, feature]
    positive_portion = (subj_feature_shap_value_over_time > 0)
    negative_portion = (subj_feature_shap_value_over_time < 0)

    pos_function = subj_feature_shap_value_over_time.copy()
    pos_function[negative_portion] = 0

    neg_function = subj_feature_shap_value_over_time.copy()
    neg_function[positive_portion] = 0

    if feature in selected_features_by_impact:
        # set value to zero before the significant timestep
        important_ts_idx = np.where(selected_features_by_impact == feature)[0]
        pos_function[:significant_timesteps[important_ts_idx][0] + 1] = 0
        neg_function[:significant_timesteps[important_ts_idx][0] + 1] = 0

    if feature in selected_positive_features:
        feature_color = positive_color_palette[pos_count]
        pos_count += 1
    elif feature in selected_negative_features:
        feature_color = negative_color_palette[neg_count]
        neg_count += 1
    elif feature in selected_negative_features_by_impact:
        feature_color = negative_color_palette[neg_count]
        neg_count += 1
    elif feature in selected_positive_features_by_impact:
        feature_color = positive_color_palette[pos_count]
        pos_count += 1
    feature_color_dict[feature] = feature_color

    if np.any(pos_function):
        positive_feature = pos_baseline + k * pos_function
        ax.fill_between(timestep_axis , pos_baseline, positive_feature, color=feature_color, alpha=alpha)
        pos_baseline = positive_feature

    if np.any(neg_function):
        negative_feature = neg_baseline + k * neg_function
        ax.fill_between(timestep_axis, negative_feature, neg_baseline, color=feature_color, alpha=alpha)
        neg_baseline = negative_feature

    # add a legend entry for the feature fill
    ax.scatter([], [], color=feature_color, alpha=alpha, label=feature_names[feature],marker="s", s=200)


# marking inflection points
for feature in set(selected_features_by_impact):
    important_ts_idx = np.where(selected_features_by_impact == feature)[0]
    for ts_idx in important_ts_idx:
        # downward inflection point
        if subj_pred_over_ts[significant_timesteps[ts_idx]] > subj_pred_over_ts[significant_timesteps[ts_idx] + 1]:
            marker = 'v'
            marker_y_level = pos_baseline[significant_timesteps[ts_idx]] + 0.005
            text_y_level = marker_y_level + 0.01
        # upward inflection point
        else:
            marker = '^'
            marker_y_level = neg_baseline[significant_timesteps[ts_idx]] - 0.005
            text_y_level = marker_y_level - 0.015

        ax.scatter(significant_timesteps[ts_idx], marker_y_level, color=feature_color_dict[feature], s=100, marker=marker, alpha=1, edgecolors='white')
        # insert a label on the plot
        ax.text(significant_timesteps[ts_idx]+ 0.01, text_y_level, feature_names[feature], fontsize=12, color='black')


ax.legend(fontsize='x-large')
ax.set_title(f'Predictions for subject {subj} of test set along time', fontsize=20)
ax.set_xlabel('Time from admission (hours)', fontsize=15)
ax.set_ylabel('Probability of favorable outcome', fontsize=15)

plt.plot()

## Plot contribution of a specific feature

In [ ]:
np.array(original_features)

In [ ]:
selected_features = ["median_mean_blood_pressure", "median_diastolic_blood_pressure", "median_systolic_blood_pressure"]


In [ ]:
selected_features_idx = [np.where(np.array(original_features) == selected_feature)[0][0] for selected_feature in selected_features]
selected_features_idx

In [ ]:
cumulative_shap_values_over_time = np.array([shap_values_over_time[ts][0].sum(axis=1) for ts in range(n_time_steps)])
subj_pred_over_ts = predictions_over_time[:,subj]

fig3 = plt.figure(figsize=(15,10))

k=1
alpha=0.3

timestep_axis = np.array(range(n_time_steps))

positive_color_palette = sns.color_palette("mako", n_colors=len(selected_positive_features))
negative_color_palette = sns.color_palette("flare_r", n_colors=len(selected_negative_features))

timestep_axis = np.array(range(n_time_steps))
ax = sns.lineplot(x=timestep_axis, y=subj_pred_over_ts, label='probability', linewidth = 2)
ax2 = ax.twinx()


pos_baseline = subj_pred_over_ts
neg_baseline = subj_pred_over_ts
pos_count, neg_count = 0, 0
for i, feature in enumerate(selected_features_idx):
    subj_cumulative_shap_value_over_time = cumulative_shap_values_over_time[:, subj, feature]
    positive_portion = (subj_cumulative_shap_value_over_time > 0)
    negative_portion = (subj_cumulative_shap_value_over_time < 0)

    pos_function = subj_cumulative_shap_value_over_time.copy()
    pos_function[negative_portion] = 0

    neg_function = subj_cumulative_shap_value_over_time.copy()
    neg_function[positive_portion] = 0

    if sum(subj_cumulative_shap_value_over_time) > 0:
        feature_color = positive_color_palette[pos_count]
        pos_count += 1
    else:
        feature_color = negative_color_palette[neg_count]
        neg_count += 1

    positive_feature = pos_baseline + k * pos_function
    ax.fill_between(timestep_axis, pos_baseline, positive_feature, color=feature_color, alpha=alpha, label=features[feature])
    pos_baseline = positive_feature

    negative_feature = neg_baseline + k * neg_function
    ax.fill_between(timestep_axis, negative_feature, neg_baseline, color=feature_color, alpha=alpha)
    neg_baseline = negative_feature

    sns.scatterplot(y=features[feature], x='index', data=non_norm_subj_df.reset_index(), ax=ax2, legend=False, color=feature_color)

ax.legend(fontsize='x-large')

ax.set_title(f'Predictions for subject {subj} of test set along time', fontsize=20)
ax.set_xlabel('Time from admission (hours)', fontsize=15)
ax.set_ylabel('Probability of favorable outcome', fontsize=15)

plt.plot()

In [ ]:
fig4 = plt.figure(figsize=(15, 12))
plt.subplots_adjust(hspace=0.2)
plt.suptitle("Selected features", fontsize=18, y=0.99, x=0.52, horizontalalignment='center')

# set number of columns (use 3 to demonstrate the change)
ncols = 3
# calculate number of rows
nrows = len(selected_features_idx) // ncols + (len(selected_features_idx) % ncols > 0)

pos_count = 0
neg_count = 0
# loop through the length of features and keep track of index
for n, feature in enumerate(selected_features_idx):
    # add a new subplot iteratively using nrows and cols
    ax = plt.subplot(nrows, ncols, n + 1)

    if sum(subj_cumulative_shap_value_over_time) > 0:
        feature_color = positive_color_palette[pos_count]
        pos_count += 1
    else:
        feature_color = negative_color_palette[neg_count]
        neg_count += 1
    sns.lineplot(y=features[feature], x='index', data=non_norm_subj_df.reset_index(), color=feature_color, ax=ax)

    ax.set_title(features[feature])
plt.tight_layout()

In [ ]:
shap_values_over_time[-1][0].shape

In [ ]:
import shap

auto=False

X_test_2D = test_X_np.reshape(-1,n_channels)
shap_values_2D = shap_values_over_time[-1][0].reshape(-1,n_channels)
x_test_2d = pd.DataFrame(data=X_test_2D, columns = features)

for n, feature in enumerate(selected_features_idx):
    if auto == True:
        # automatic choice of interaction
        shap.dependence_plot(features[feature], shap_values_2D, x_test_2d)
    else:
        shap.dependence_plot(features[feature], shap_values_2D, x_test_2d, interaction_index="median_NIHSS")